In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import shutil
import time

# Step 1 - Scraping

## NASA Mars News

In [7]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'



In [8]:
response = requests.get(url)

In [9]:
soup = bs(response.text, 'html.parser')

In [10]:
results = soup.find('div', class_ = 'slide')
print(results.prettify())

<div class="slide">
 <div class="image_and_description_container">
  <a href="/news/8426/nasa-garners-7-webby-award-nominations/">
   <div class="rollover_description">
    <div class="rollover_description_inner">
     Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.
    </div>
    <div class="overlay_arrow">
     <img alt="More" src="/assets/overlay-arrow.png"/>
    </div>
   </div>
   <img alt="NASA Garners 7 Webby Award Nominations" class="img-lazy" data-lazy="/system/news_items/list_view_images/8426_Webby2019-320x240.jpg" src="/assets/loading_320x240.png"/>
  </a>
 </div>
 <div class="content_title">
  <a href="/news/8426/nasa-garners-7-webby-award-nominations/">
   NASA Garners 7 Webby Award Nominations
  </a>
 </div>
</div>



In [11]:
content_title = soup.find_all('div', class_ = "content_title")
title = []

for ct in content_title:
    if (ct.a):
         if (ct.a.text):
                title.append(ct.text)


    title_text = title[0]
    title_text = title_text.replace("\n\n", "")
    
print(title_text)

NASA Garners 7 Webby Award Nominations


In [12]:
description = soup.find('div', class_ = "rollover_description_inner")
print(description.prettify())

<div class="rollover_description_inner">
 Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.
</div>



In [57]:
desc = []
for dc in description:
    if (dc):
        desc.append(dc)
        
    description_ = dc.replace("\n","")
print(description_)
        


Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.


## JPL Mars Space Images - Featured Image

In [61]:

# Use splinter to navigate the site and find the image url 
# for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.
# https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

/usr/local/bin/chromedriver


In [62]:
# Visit Mars Space Images through splinter module
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_image)

In [63]:
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[3]/a/div/div[2]/img"

In [65]:
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [66]:
html_img =browser.html
soup = bs(html_img, "html.parser")
img_url = soup.find('img', class_='fancybox-image')["src"]

In [67]:
from urllib.parse import urlsplit
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_image))
print(base_url)

https://www.jpl.nasa.gov/


In [68]:
featured_image_url = base_url + img_url
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23140_hires.jpg


## Mars Weather

In [60]:
# Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called mars_weather.
from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

/usr/local/bin/chromedriver


In [61]:
html_weather = browser.html
soup = bs(html_weather, "html.parser")
mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

In [62]:
print(mars_weather)

InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF)
winds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph)
pressure at 7.30 hPapic.twitter.com/bRsLlzn4M0


In [63]:
#remove hPapic.twitter.com/kmh5FXGbBL

In [64]:
mars_weather = mars_weather.split('hPa')[0]

In [68]:
import re
patterns = ['r"http\S+"', 'r"\xa0pic\S+"', 'r"pic\S+"', 'r"pbs\S+"']
for pattern in patterns:
    mars_weather = re.sub(pattern, "", mars_weather).strip()

InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF)
winds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph)
pressure at 7.30


## Mars Facts

In [37]:
url_facts = "https://space-facts.com/mars/"

In [38]:
table = pd.read_html(url_facts)

In [31]:
table_df = pd.DataFrame(table[0])
table_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [32]:
table_df.columns=['Index', 'Values']

In [33]:
table_df = table_df.set_index('Index')

In [34]:
table_df

,Values
Index,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [35]:
table_df = table_df.to_html()
table_df.replace("\n", " ")

'<table border="1" class="dataframe">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Values</th>     </tr>     <tr>       <th>Index</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

## Mars Hemispheres

1. Visit the USGS Astrogeology site here to obtain high resolution 
   images for each of Mar's hemispheres.
    
2. You will need to click each of the links to the hemispheres
   in order to find the image url to the full resolution image.
    
3. Save both the image url string for the full resolution 
   hemisphere image, and the Hemisphere title containing 
   the hemisphere name. Use a Python dictionary to store 
   the data using the keys img_url and title.
    
4. Append the dictionary with the image url string and the 
   hemisphere title to a list. This list will contain 
   one dictionary for each hemisphere.

In [94]:
from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

/usr/local/bin/chromedriver


In [137]:
# scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
h_url = requests.get(mars_hemisphere_url)


In [144]:
# for loop to get path for each imgs
# xpath1 = "//*[@id='product-section']/div[2]/div[i]/a/img"
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemisphere_url)
time.sleep(1)
 

xpath1 = []
hemi_url = []

# Get path for each hemisphere website
for i in range(1,5):
    x = f"//*[@id='product-section']/div[2]/div[{i}]/a/img"

    
    
    results = browser.find_by_xpath(x)
    img = results[0]
    img.click()
    time.sleep(1)

    xpath3 = "//*[@id='wide-image']/div/ul/li/a"
    results = browser.find_by_xpath(xpath3)
    img = results[0]
    img.click()
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    url = soup.find("img")["src"]
    
    browser.windows.current = browser.windows[-1]
    hemi_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()

    
    
    url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url_hemisphere)

In [134]:
# scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemi_dicts = []

for i in range(1,9):
    hemi_dict = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    hemispheres_html = browser.html
    hemispheres_soup = bs(hemispheres_html, 'html.parser')
    hemi_name_links = hemispheres_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemi_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemi_img_soup = bs(hemi_img_html, 'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']

    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(hemi_img_path)
    hemi_dict['img_url'] = hemi_img_path

    hemi_dicts.append(hemi_dict)


Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg


WebDriverException: Message: unknown error: Element <a href="/search/map/Mars/Viking/schiaparelli_enhanced" class="itemLink product-item">...</a> is not clickable at point (22, 335). Other element would receive the click: <div class="item">...</div>
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=73.0.3683.68 (47787ec04b6e38e22703e856e101e840b65afe72),platform=Mac OS X 10.14.4 x86_64)


In [145]:
from splinter import Browser
!which chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

/usr/local/bin/chromedriver


In [146]:
hemisphere_url = "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"
browser.visit(hemisphere_url)


In [147]:
html = browser.html
soup = bs(html, 'html.parser')
url = soup.find("img")["src"]

In [148]:
url

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

In [81]:
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[3]/a/div/div[2]/img"
results = browser.find_by_xpath(xpath)
img = results[0]
img.append()

ElementDoesNotExist: no elements could be found with xpath "//*[@id="page"]/section[3]/div/ul/li[3]/a/div/div[2]/img"

In [104]:
# Obtaining each url
url_hemisphere2 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemisphere2)
hemi_url = []
html = browser.html
soup = bs(html, 'html.parser')

img_url = soup.find("img")["src"]
hemi_url.append({"img_url": img_url})
#hemi_url.append({"title": title, "img_url": img_url})


#/html/body/div/span[@class='regular_price']


In [105]:
hemi_url

[{'img_url': '/images/usgs_logo_main_2x.png'}]

In [ ]:
# Use the requests library to download and save the image from the `img_url` above
response = requests.get(img_url, stream=True)
with open('img.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)